<a href="https://colab.research.google.com/github/akshaypt7/energy_predictor/blob/main/Energy_predictor_3_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from IPython.display import display, HTML

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split

import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression


from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , mean_squared_log_error


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

from sklearn.inspection import permutation_importance, plot_partial_dependence, permutation_importance

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.preprocessing import OrdinalEncoder
from sklearn.datasets import load_boston

import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor

# Enable multiple cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import pickle


In [ ]:
df_merge= pd.read_pickle('/content/drive/MyDrive/Energy_predictor/df_merge_filled_na.pkl')

In [ ]:
df_merge.reset_index(inplace=True)

This is a very big dataset, for this to work we need to sample the data.

- we can create a model based on the sample data from this data, we can also create dataset based on meter_types and site_ids.

- Then we can compare which model is giving us the best score for the metric

In [ ]:
# from univ.ai homework 2

def sample(X, y, n):
    '''random samples'''
    num_row = X.shape[0]
    indices = np.random.randint(num_row, size=n)
    return X[indices, :], y[indices]

The above function maynot give us the sample with similar distribution of the parent dataset.

Instead we can use the train_test_split with stratify so that we can the test set in that as the main dataset

Calculating the size of the sample needed.

We can use 5,00,000 Samples

In [ ]:
no_of_samples = 500000

size = no_of_samples/df_merge.shape[0]
size

0.024732762501174805

In [ ]:
X = df_merge.drop('meter_reading', axis=1)
y= df_merge['meter_reading']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify= X['meter'], 
                                                    test_size= size)

In [ ]:
df_merge  = X_test

In [ ]:
df_merge['meter_reading'] = y_test

In [ ]:
# df_merge.reset_index(inplace=True)

In [ ]:
y =  np.log(df_merge['meter_reading'])

In [ ]:
y= y.replace([-np.inf],0)

In [ ]:
y = np.where(y<0,0,y)

In [ ]:
# sns.distplot(df_merge['meter_reading'])

Log of meter_reading created small negative values, we fill those values with 0

In [ ]:
y = np.where(y<0,0,y)

In [ ]:
# sns.distplot(y)

#### RMSE

In [ ]:
def rmse(y_true, y_pred):
    return np.sqrt(np.mean(np.square(y_true - y_pred)))
    

#### Encoding

In [ ]:
categorical_features = ['year_built','primary_use'] # we need to add primary_use here, we have already scaled primary_use
continous_features = [
        'square_feet', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'sea_level_pressure', 'wind_direction', 'wind_speed']


How do we do the pipeline

- first we write the individual encoders
- 

In [ ]:
cat_transformer = OrdinalEncoder()
cont_transformer = StandardScaler()
preprocessor = ColumnTransformer(transformers=[('cat', cat_transformer, categorical_features), ('cont', cont_transformer, continous_features)], remainder='passthrough')
model = Pipeline(steps=[('pre_processor', preprocessor)])

In [ ]:
X = df_merge.drop(['meter_reading','datetime','timestamp'],axis = 1)
y = y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify= X['meter'], 
                                                    test_size= 0.3)

NameError: ignored

In [ ]:
model_pipe = Pipeline(steps= [( 'modeling' , model), ('rf', RandomForestRegressor(warm_start=True, oob_score=True, n_jobs=1, random_state=66, max_features=0.33))])

In [ ]:
param_grid_pipeline = { "rf__n_estimators": range(300,600, 200), "rf__max_features": [0.2,0.5,0.7], "rf__min_samples_leaf" : [5,10,20,25]}

In [ ]:
rf_gs_pipeline = GridSearchCV(model_pipe , param_grid=param_grid_pipeline, cv=4)

In [ ]:
rf_gs_pipeline.fit(X_train,y_train)

In [ ]:
rf_model_filename_gs = '/content/drive/MyDrive/Energy_predictor/RF_model_GS.pickle'

In [ ]:
pickle.dump(rf_gs_pipeline,open(rf_model_filename_gs,'wb'))

In [ ]:
y_pred = rf_gs_pipeline.predict(X_test)

In [ ]:
rmse(y_test,y_pred)

In [ ]:
rf_simple.score(X_test,y_test)  

##### Simple random forest

In [ ]:
rf_sim = RandomForestRegressor(warm_start=True, oob_score=True, n_jobs=1, random_state=66, max_features=0.33, n_estimators=200, min_samples_leaf=20)

In [ ]:
rf_simple = Pipeline(steps= [( 'modeling' , model), ('rf', rf_sim)])

In [ ]:

# rf_simple.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('modeling',
                 Pipeline(memory=None,
                          steps=[('pre_processor',
                                  ColumnTransformer(n_jobs=None,
                                                    remainder='passthrough',
                                                    sparse_threshold=0.3,
                                                    transformer_weights=None,
                                                    transformers=[('cat',
                                                                   OrdinalEncoder(categories='auto',
                                                                                  dtype=<class 'numpy.float64'>),
                                                                   ['year_built',
                                                                    'primary_use']),
                                                                  ('cont',
                                  

In [ ]:
rf_model_filename = '/content/drive/MyDrive/Energy_predictor/RF_model_simple.pickle'

In [ ]:
# pickle.dump(rf_simple,open(rf_model_filename,'wb'))

In [ ]:
y_pred = rf_simple.predict(X_test)

In [ ]:
rmse(y_test,y_pred)

0.9907717894390861

In [ ]:
rf_simple.score(X_test,y_test)  

0.7944886651876198

In [ ]:
rf_simple = pickle.load(open(rf_model_filename, 'rb'))

In [ ]:
# rf_gs_pipeline = pickle.load(open(rf_model_filename,'rb'))

***Important Note : When using pipeline for grid_search , we need to add prefix of the 'model__' along with the paramaters (param_grid). - refere here - https://stackoverflow.com/questions/41899132/invalid-parameter-for-sklearn-estimator-pipeline***